# Subspace Digital Twin, Initial Conditions Run

*Shawn Anderson, January 2024*

In this notebook, we examine medianl behavior over the first 90 days.

## Part 1. Dependences & Set-up

Autoreload modules while developing.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import pandas as pd
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

import hvplot.pandas
hvplot.extension('bokeh')

from bokeh.models import HoverTool
import holoviews as hv

from bokeh.palettes import Turbo256, Category20

## Part 2. Load Simulation Data

Load the simulation results data.

In [2]:
sim_df = pd.read_pickle(
    "../data/simulations/reference_subsidy_sweep-2024-01-10_12-04-38.pkl.gz"
).drop(['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'], axis=1)

In [3]:
sim_df.head(5)

,days_passed,blocks_passed,circulating_supply,user_supply,earned_supply,issued_supply,earned_minus_burned_supply,total_supply,sum_of_stocks,block_utilization,dsf_relative_disbursal_per_day,reward_issuance_balance,other_issuance_balance,operators_balance,nominators_balance,holders_balance,farmers_balance,staking_pool_balance,fund_balance,burnt_balance,nominator_pool_shares,operator_pool_shares,block_reward,history_size,space_pledged,allocated_tokens,buffer_size,reference_subsidy,average_base_fee,average_priority_fee,average_compute_weight_per_tx,average_transaction_size,transaction_count,average_compute_weight_per_bundle,average_bundle_size,bundle_count,compute_fee_volume,storage_fee_volume,rewards_to_nominators,run,average_compute_weight_per_budle,label,environmental_label,max_credit_supply
0,0,0.0,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,NaN,0,0,0.0,0.0,0.00000,0.0,0.0,0.0,256,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,1,NaN,constant-single-component,standard,3000000000
14,1,14400.0,13.698630,13.698630,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.684932,13.013699,0.0,0.000000,0.0,0.0,0.0,13.698630,2038498852864,1572944000000,0.0,60045568.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.000000,0.000000,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
28,2,28800.0,27.363014,27.397260,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,1.368151,25.994863,0.0,0.034247,0.0,0.0,0.0,13.698630,4076997705728,3145888000000,0.0,120091136.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.342466,0.342466,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
42,3,43200.0,41.201036,41.303690,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,2.060052,39.140985,0.0,0.102654,0.0,0.0,0.0,13.906430,6115764994048,4718832000000,0.0,45918976.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.684075,0.684075,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
56,4,57600.0,55.211744,55.417401,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000745,0.000745,2.829882,52.380372,0.0,0.205657,0.0,0.0,0.0,14.113711,8154263846912,6291776000000,0.0,105964544.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.955515,1.030026,0.0,1,1.000000e+10,constant-single-component,standard,3000000000


Simulation Runs.

In [4]:
sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='Days').head()

,run,label,environmental_label,Days
0,1,constant-single-component,standard,185
1,2,hybrid-single-component,standard,185
2,3,hybrid-two-components,standard,185


### Coloring Metrics
Use a constant mapping from columns to colors

In [5]:
color_palette = Category20
# columns_to_color = sorted(list(set(sim_df.columns) - {'environmental_label', 'label', 'run', 'blocks_passed', 'days_passed'}))
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}


sim_df.count().to_frame().T.hvplot.bar(y=columns_to_color, color=[column_colors[c] for c in columns_to_color], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

:Bars   [index,Variable]   (value)

In [6]:
def snake_to_title(s):
    """Utility function used for printing chart titles and labels as Title Case.
    Example:
    snake_to_caps('snake_case')
    >>> 'Snake Case'
    """
    
    return ' '.join(word.capitalize() for word in s.split('_'))

def fan_chart_quantile_median(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of median for a series."""

    # min, max, median
    fan_df = df.groupby('days_passed')[column].agg(['min', 'max', 'median'])

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Fan Chart', ylabel=f'{column}_min_max_median')

    # Median curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    median_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Median', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return median_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='min', y2='max', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * median_chart
    return chart


def fan_chart_quantile(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of quantile for a series."""

    # 25%, 50%, 75%
    fan_df = df.groupby('days_passed')[column].quantile([0.25, 0.5, 0.75]).unstack().rename(columns={0.50:'median', 0.25:'0.25',0.75:'0.75'})

    # return fan_df

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Quantile Fan Chart', ylabel=f'{column}_quantile')

    # Quantile curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    quatile_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Quantile', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return quatile_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='0.25', y2='0.75', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * quatile_chart
    return chart


In [7]:
system_balances = ['other_issuance_balance', 'reward_issuance_balance']
agent_balances = [
    'farmers_balance',
    'operators_balance',
    'user_supply',
    'nominators_balance',
]
agent_pool_balances = ['staking_pool_balance']
protocol_treasury_balances = ['fund_balance']
other_balances = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances + agent_balances + agent_pool_balances + protocol_treasury_balances) )
supply_columns = list({c for c in sim_df.columns if 'supply' in c} - {'max_credit_supply', 'issued_supply', 'total_supply'})
balance_columns = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances))

In [8]:
box_df = sim_df.set_index(['days_passed', 'label'])[balance_columns]
describe_df = box_df.describe().drop('count')
describe_df

,fund_balance,staking_pool_balance,holders_balance,nominators_balance,burnt_balance,operators_balance,farmers_balance
mean,821.217333,37.477973,258.681382,54.217781,0.140061,18.262442,2530.585467
std,817.950764,43.648477,171.751254,49.514348,0.196467,13.872263,1652.077357
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,129.658052,2.587962,118.239567,11.047337,0.002493,6.271811,1181.400399
50%,557.549602,19.559475,221.116090,41.209783,0.043680,15.607416,2160.805712
75%,1226.934664,57.718636,412.985247,86.268773,0.205256,30.215930,4017.654500
max,2944.939900,164.007019,580.032701,169.498502,0.805923,45.187272,5644.475529


In [9]:
describe_labels_df = box_df.groupby('label').apply(lambda label: label.describe().drop('count'))
describe_labels_df

fund_balance  staking_pool_balance  \
label                                                                
constant-single-component mean    420.979365             18.948390   
                          std     374.139104             20.048295   
                          min       0.000000              0.000000   
                          25%      75.260983              1.402548   
                          50%     325.223007             11.113450   
                          75%     716.559421             32.814201   
                          max    1218.857093             67.032035   
hybrid-single-component   mean   1021.336139             46.742765   
                          std     903.556742             49.074055   
                          min       0.000000              0.000000   
                          25%     186.694333              3.704750   
                          50%     792.406737             27.810490   
                          75%    1736.031131             80.862353   
                          max    2944.873868            164.007019   
hybrid-two-components     mean   1021.336495             46.742765   
                          std     903.557506             49.074055   
                          min       0.000000              0.000000   
                          25%     186.694333              3.704750   
                          50%     792.406737             27.810490   
                          75%    1736.031131             80.862353   
                          max    2944.939900            164.007019   

                                holders_balance  nominators_balance  \
label                                                                 
constant-single-component mean       133.061617           27.579172   
                          std         72.218847           22.257287   
                          min          0.000000            0.000000   
                          25%         73.525110            6.266923   
                          50%        142.552785           24.198728   
                          75%        196.733257           46.764043   
                          max        239.257344           69.822894   
hybrid-single-component   mean       321.475337           67.537068   
                          std        172.894560           53.926146   
                          min          0.000000            0.000000   
                          25%        179.327920           16.016093   
                          50%        344.317833           59.648747   
                          75%        473.819400          114.070529   
                          max        575.459992          169.491898   
hybrid-two-components     mean       321.507193           67.537103   
                          std        172.941877           53.926213   
                          min          0.000000            0.000000   
                          25%        179.327920           16.016093   
                          50%        344.317833           59.648747   
                          75%        473.819400          114.070529   
                          max        580.032701          169.498502   

                                burnt_balance  operators_balance  \
label                                                              
constant-single-component mean       0.070329           9.318630   
                          std        0.090781           6.041727   
                          min        0.000000           0.000000   
                          25%        0.001339           3.737748   
                          50%        0.024212           9.789290   
                          75%        0.114579          14.764573   
                          max        0.326650          18.682944   
hybrid-single-component   mean       0.174926          22.734330   
                          std        0.224324          14.530433   
                          min        0.

In [10]:
describe_difference_df = pd.DataFrame(describe_labels_df.values - pd.concat([describe_df for i in range(sim_df['label'].nunique())]).values, columns=describe_labels_df.columns, index=describe_labels_df.index)
df = describe_difference_df

def log_scale(val, max_abs_log):
    """ Apply logarithmic scaling to a value. """
    if val == 0:
        return 0
    else:
        return np.sign(val) * np.log(abs(val) + 1) / max_abs_log

def color_scale(val):
    max_abs_val = df.abs().max().max()
    max_abs_log = np.log(max_abs_val + 1)

    scaled_val = log_scale(val, max_abs_log)

    if scaled_val < 0:
        intensity = int(255 * (1 + scaled_val))  # More negative, more red
        return f'background-color: rgb(255, {intensity}, {intensity})'
    elif scaled_val > 0:
        intensity = int(255 * (1 - scaled_val))  # More positive, more green
        return f'background-color: rgb({intensity}, 255, {intensity})'
    else:
        return 'background-color: rgb(255, 255, 255)'

header_styles = [{
    'selector': f'th.col_heading.level0.col{i}',
    'props': [('background-color', column_colors.get(col))]
} for i, col in enumerate(df.columns)]

df.columns.name = 'balance'

describe_difference_df_styled = df.style.map(color_scale).set_table_styles(header_styles)
describe_difference_df_styled

In [11]:
box_df_melted = box_df.reset_index().drop('days_passed',axis=1).melt(id_vars=['label'])

violin_list = [label.hvplot.violin(y='value', by='variable', c='variable', legend='top_right', width=1200, height=500, title=f'SSC Balances {name}', cmap=column_colors, ylim=(0,box_df.max().max()*0.75)) for name, label in box_df_melted.groupby('label')]

# Combine plots into a single column layout
layout = hv.Layout(violin_list).cols(1)

layout

:Layout
   .Violin.I   :Violin   [variable]   (value)
   .Violin.II  :Violin   [variable]   (value)
   .Violin.III :Violin   [variable]   (value)

In [12]:
line_list = [hv.Overlay([fan_chart_quantile(label, column) for column in label.columns if 'balance' in column]).opts(title=f'SSC Balances {name}') for name, label in box_df.reset_index().groupby('label')]
layout = hv.Layout(line_list).cols(1)
layout

:Layout
   .Overlay.I   :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
   .Overlay.II  :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
   .Overlay.III :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)

In [25]:
violin_list = [variable.hvplot.violin(y='value', by='label', color=column_colors[name], width=1200, height=500, title=f'SSC Balances {name}', ylim=(0,variable.max()['value'].max())) for name, variable in box_df_melted.groupby('variable')]

layout = hv.Layout(violin_list).cols(1).opts(shared_axes=False)

layout

:Layout
   .Violin.I   :Violin   [label]   (value)
   .Violin.II  :Violin   [label]   (value)
   .Violin.III :Violin   [label]   (value)
   .Violin.IV  :Violin   [label]   (value)
   .Violin.V   :Violin   [label]   (value)
   .Violin.VI  :Violin   [label]   (value)
   .Violin.VII :Violin   [label]   (value)

In [33]:
line_list = [variable.hvplot.line(x='days_passed', by='label', y='value', title=name) for name, variable in box_df.reset_index().melt(id_vars=['label', 'days_passed']).groupby('variable')]

layout = hv.Layout(line_list).cols(2).opts(shared_axes=False)
layout

:Layout
   .NdOverlay.I   :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.II  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.III :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.IV  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.V   :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.VI  :NdOverlay   [label]
      :Curve   [days_passed]   (value)
   .NdOverlay.VII :NdOverlay   [label]
      :Curve   [days_passed]   (value)